## References
### Wikipedia
[Iterated Function Systems](https://en.wikipedia.org/wiki/Iterated_function_system)

[Chaos Game](https://en.wikipedia.org/wiki/Chaos_game)

[L-systen](https://en.wikipedia.org/wiki/L-system)

[Barnsley Fern](https://en.wikipedia.org/wiki/Barnsley_fern)

### Python Packages

[turtle graphics](https://docs.python.org/3/library/turtle.html)<br>
Python 3.10 lib/turtle.py documentation

[The Chaos Game](https://beltoforion.de/en/recreational_mathematics/chaos_game.php)<br>
An implementation of the Chaos Game using polygons

## Mathematica
[SubstitutionSystem](https://reference.wolfram.com/language/ref/SubstitutionSystem.html)

In [19]:
from music21 import stream, interval, corpus, instrument, clef, key
from music21 import converter, note, chord, environment, duration, pitch
from music21.meter import TimeSignature
import pandas as pd

class MusicScale(object):
    def __init__(self,  resource_folder ="/Compile/dwbzen/resources/music", \
                 scale_name='Major', root_note=note.Note('C4'), key=key.Key('C')):
        
        self.scales_df = pd.read_json(resource_folder + "/commonScaleFormulas.json", orient='records').transpose()
        self.scale = self.scales_df.loc[scale_name]
        self.root = root_note
        self.scale_name = scale_name
        self.formula = self.scale['formula']
        self.key = key
        self.notes_stream = None
        self.scale_notes = self.get_scale_notes(root_note)
        self.scale_notes_names = [x.nameWithOctave for x in self.scale_notes]
        self.range_notes = self.get_range_notes(root_note)
        self.range_notes_names = [x.nameWithOctave for x in self.range_notes]
        
    def get_scale_notes(self, start_note=None):
        '''The notes of the configured scale spanning a single octave, for example 'C4' to 'C5'
            Note that the top note is the same as the bottom note but an octave higher.
            Returns: a [Note]
        '''
        ns = stream.Stream()
        ns.append(self.key)
        if start_note is None:
            n = self.root
        else:
            n = start_note
        ns.append(n)
        for i in range(len(self.formula)):
            n = n.transpose(self.formula[i], inPlace=False)
            ns.append(n)
        return [x for x in ns.notes]
    
    def get_range_notes(self, start_note=None, start_octave=0, end_octave=8):
        '''For example, with default arguments notes (Major scale) are C0,D0,...B7,C8
        
        '''
        if start_note is None:
            n = self.root
            start_note = self.root
        else:
            n = start_note
        notes_list = []
        for octave in range(start_octave, end_octave):
            for n in self.scale_notes[:-1]:
                new_note = note.Note(n.name+str(octave))
                notes_list.append(new_note)
        new_note = note.Note(start_note.name+str(end_octave))
        notes_list.append(new_note)
        return notes_list
    
    def get_note(self, scale_degrees:int=0, from_note=None) -> note.Note:
        '''Gets a new Note that is the given scale_degrees away from a given Note.
            Arguments:
                scale_degrees - any integer, the sign dictates the direction - up or down
                from_note - the starting note, default is the scale root note
            Returns: a new Note
            Note if from_note is not present in the full range of notes (self.range_notes), the from_note is returned.
        '''
        if from_note is None:
            from_note = self.root

        if from_note.nameWithOctave in self.range_notes_names:
            ind = self.range_notes_names.index(from_note.nameWithOctave) + scale_degrees
            return self.range_notes[ind]
        else:
            return from_note


In [2]:
mscale = MusicScale(scale_name="Harmonic Minor",root_note=note.Note('C5'))
print(mscale.scale_notes)
print(mscale.formula)
n = mscale.scale_notes[2]
print(n.nameWithOctave)
rn = mscale.get_range_notes(start_octave=4, end_octave=6)
print([x.nameWithOctave for x in rn])

rn = mscale.get_range_notes()
e4 = rn[30]
anote = note.Note("E-4", quarterLength=2)
mscale.get_note(3, anote).nameWithOctave

'G#4'

[<music21.note.Note C>, <music21.note.Note D>, <music21.note.Note E->, <music21.note.Note F>, <music21.note.Note G>, <music21.note.Note G#>, <music21.note.Note B>, <music21.note.Note C>]
[2, 1, 2, 2, 1, 3, 1]
E-5
['C4', 'D4', 'E-4', 'F4', 'G4', 'G#4', 'B4', 'C5', 'D5', 'E-5', 'F5', 'G5', 'G#5', 'B5', 'C6']


In [3]:
mscale.get_note(-10).nameWithOctave

'G3'

## Combined Rules

The previous code has two  rule sets for pitch and duration that were applied independently: the pitch rules first, then the duration rules on the result.  This isn't quite what I had envisioned.

This version combines pitch and duration substitution into a single rule. The downside is the rules are more complex and can no longer be represented by a 2 characters (pitch) or number (duration).<br>
However it is more intuitive and easier to construct logically. For example a rule might be<br>
**replace a given note with 4 notes with pitches 0, +1, -2, +1, and duration half the existing duration**

The pitch rule is expressed as an interval from the previous pitch as before.<br>
The duration rule is expressed as a multiple of the duration of the previous note.

In addition each substitution has optional pre- and post- processing rule(s). The duration post-processing rule for example
would check if the resulting duration is less than, say, a 16th note (0.25 quarterLength) and if so set to 1 quarterLength.<br>
The RuleSet class then will consist of the following:
1. Pre-processing rules expressed as functions.
2. Pitch, duration substitutions as an ordered pair: (pitch, duration). For example ('0+1-2+1', 0.5)
3. Post-processing rules expressed as function(s).

Rule application is recursive: each new generation is created by applying the rule set to the previous generation.

In [3]:
import re

class RuleSet(object):

    def __init__(self, substitution_rules:dict, rules=None, splitter:str=None):
        '''
        Each substitution is a dict. The key is a regular expression string used to match an input string,
        for example: (interval)/(quarterLength-multiplier)
        The value is the replacement. The re groups are named and can be referenced in the replacement.
        splitter is a regular expression, analagous to a string splitter, that will split the
        substitution result into named components.
        
        Sample substitutions:
           sub1 = {r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}
           sub2 = {r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/<duration>', '1/0.5']}
           splitter = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
        '''
        self.rules = rules
        self.substitutions = []
        self.substitution_rules = substitution_rules

        #
        # compile the substitution rules regular expressions
        #
        for k in self.substitution_rules.keys():
            pattern = re.compile(k)
            replacement = self.substitution_rules[k]
            self.substitutions.append({'pattern':pattern, 'replacement':replacement})
        
        self.pre_processing = None
        self.post_processing = None
        self.command_rules = None
        if splitter is not None:
            self.splitter = re.compile(splitter)
        if rules is not None:
                if 'preProcessing' in rules:
                    self.pre_processing = rules['preProcessing'] 
                if 'postProcessing' in rules:
                    self.post_processing = rules['postProcessing']
                if 'commands' in rules:
                    self.command_rules = rules['commands']
                

In [15]:
#
# test RuleSet
#
substitution_rules = {\
        r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0'], \
        r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/<duration>', '1/0.5']\
}
ruleSet = RuleSet(substitution_rules)
print(f'substitutions: {ruleSet.substitutions}\n')
for subs in ruleSet.substitutions:
    print(f'sub: {subs}')
    print(f' pattern: {subs["pattern"]}\n replacement: {subs["replacement"]}\n' )

substitutions: [{'pattern': re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}, {'pattern': re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/<duration>', '1/0.5']}]

sub: {'pattern': re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}
 pattern: re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)')
 replacement: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']

sub: {'pattern': re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/<duration>', '1/0.5']}
 pattern: re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)')
 replacement: ['0/<duration>', '1/0.5']



In [4]:
import re

class SubstitutionSystem(object):
    '''Base substitution system class
    '''
    def __init__(self, rule_set:RuleSet, tags:[str]=None, verbose=0):
        '''Initialize code
            Arguments:
                rule_set - a RuleSet instance
                tags - a list of tags that can appear in substitution rules, default is None
        '''
        self.rule_set = rule_set
        self.substitutions = rule_set.substitutions
        self.tags = tags
        self.verbose = verbose
    

In [5]:
import re

class MusicSubstitutionSystem(SubstitutionSystem):
    '''Generates a list representing the evolution of the substitution system with the specified rule 
        from initial condition init for t steps.
        Arguments:
            substitution_rules - 
            start - initial condition
            t - the number of steps (generations) as a range(n1, n2)
                where n1 is the starting step, and n2-1 is the ending step, 0 <= n1 <= n2
                For example, t=range(1, 4) will output the results of steps 1,2 and 3
                t=range(2, 3) step 2 only.
                t=range(2, 4) steps 2 and 3
    '''
    
    def __init__(self, rule_set:RuleSet, verbose=0):
        super().__init__(rule_set, tags=['interval', 'duration'], verbose=verbose)

    
    def apply(self, start:[str], nsteps):
        '''Apply the substitution to the start string for a given number of steps.
         The result is a command list that can be input to a music generation module.
        '''
        sub_result = start
        self.steps = nsteps
        if nsteps > 0:
            sub_result = start
            for step in range(nsteps):
                sub_result = self.apply_step(sub_result)
                if self.verbose > 0:
                    print(f'step: {step}\nresult: {sub_result}')
        return sub_result
            
        
    def apply_step(self, start:[str]):
        subst_result = []
        for subs in self.substitutions:
            pattern = subs["pattern"]
            replacement = subs["replacement"]

            # try to match each item in start (result) list
            # 
            for s in start:
                if self.verbose > 0:
                    print(f's: "{s}"    pattern: {pattern.pattern}')
                match = pattern.match(s)
                if match is not None:
                    if self.verbose > 0:
                        print(f'"{s}" matched pattern {pattern.pattern} ')
                    grp_dict = match.groupdict()
                    #
                    # now substitute
                    #
                    subst_result += replacement
                else:
                    subst_result += [s]
            if self.verbose > 0:
                print(f'step result: {subst_result} \n')
        return subst_result
                        

In [6]:
start = ['0/1.0', '+1/0.5']
substitution_rules = {\
        r'(?P<interval>[+-]?[0123])/(?P<duration>\d+\.\d+)':['0/0.5', '+3/1.0', '-2/1.0', '-1/0.5'] } #, \
        #r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/1.0', '1/0.5']   }
ruleSet = RuleSet(substitution_rules)
ss = MusicSubstitutionSystem(ruleSet, verbose=0)

In [7]:
sub_result = ss.apply(start,2)
print(f'{len(sub_result)} commands:\n{sub_result} ')

32 commands:
['0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5'] 


In [10]:
import re
# the generic match string - splitter
re_str = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
re = re.compile(re_str)
m = re.match('0/0.5')
print(m.groups())
m = re.match('+1/1.0')
print(m.groupdict()['duration'])

('0', '0.5')
1.0


## Generate a Part 
Create a class that creates a music21.Part from the commands output of MusicSubstitutionSystem

In [8]:
from music21 import stream, interval, corpus, instrument, clef, key, meter
from music21 import converter, note, chord, environment, duration, tempo, metadata
from music21.stream import Score, Part
import pandas as pd
import sys
import copy

class ScoreGen(object):
    
    def __init__(self, rule_set:RuleSet, resource_folder ="/Compile/dwbzen/resources/music", verbose=0, \
                 scale_name='Major', instrument_name='Soprano', \
                 clef=clef.TrebleClef(), key=key.Key('C'), title="Music Substiution" ):
        '''
            TODO - allow for more than 1 part (instrument). There needs to be a list of start notes,
            one for each part. instrument_name also needs to be a list.
        '''
        self.resource_folder = resource_folder
        self.scale_name = scale_name
        self.rule_set = rule_set
        self.verbose = verbose
        #
        # create score metadata
        #
        self.tempo = tempo.MetronomeMark(number=100, referent=note.Note(type='quarter'))
        self.timeSignature = meter.TimeSignature('4/4')
        
        self.score = stream.Score()
        self.score.insert(0, metadata.Metadata())
        self.score.metadata.title = title
        
        self.part = stream.Part()    # the part created from pitch rules
        self.part.partName = instrument_name
        self.part.insert(clef)
        self.part.insert(self.tempo)
        self.part.insert(instrument.Instrument(instrumentName=instrument_name))
        self.key = key
        self.part.insert(key)
        self.part.insert(self.timeSignature)
        
        self.state = dict()
        self.command_rules = rule_set.rules['commands']
        self.notes = []   # all the notes added

    def run(self, commands:[str], start_note=note.Note("C5", quarterLength=2)) -> stream.Score:
        '''The start_note must have a pitch with octave and a duration in quarterLengths
        
        ''' 
        self.musicScale = MusicScale(resource_folder=self.resource_folder, scale_name=self.scale_name, root_note=start_note)
        self.scale = self.musicScale.scale
        self.start_note = start_note
        
        self.state['note'] = start_note
        self.state['previous_note'] = None
        self.state['musicScale'] = self.musicScale
        
        self.apply_commands(commands)
        self.score.append(self.part)
        return self.score

    def apply_commands(self, commands):
        '''Creates the Part notes from the commands string by executing the associated rules
            Sample commands: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0', '+1/0.5', '0/duration', '0/1.0', '1/0.5']
        '''
        for command in commands:
            self.apply_command(command)
    
    def apply_command(self, command):
        if self.verbose > 0:
            print(f'Command: {command} -------------------------')
        group_dict = self.rule_set.splitter.match(command)
        if group_dict is None:
            return None
        interval = group_dict['interval']
        duration =  group_dict['duration']
        int_rule = self.command_rules['interval']
        dur_rule = self.command_rules['duration']
        int_rule(interval, self.state)    # apply to state['note']
        next_note = dur_rule(duration, self.state)    # apply to state['note']
        self.notes.append(next_note)
        self.part.append(next_note)
        
    

### Rules code

In [9]:
from music21 import note, duration
from music21.stream import Score

trace = False

def show(anote:note.Note):
    print(f"pitch: {anote.nameWithOctave}  duration: {anote.duration.quarterLength}")
    
def score_postprocess(ascore:Score, instrument_name:str):
    '''Checks the pitch against the range of the given instrument, adjusting if necessary.
    '''
    if trace:
        print('score_postprocess')


def score_preprocess(ascore:Score):
    if trace:
        print('score_preprocess')

def interval_rule(sd:str, state):
    anote = state['note']

    if sd == 'interval':
        scale_degrees = 0
    else:
        scale_degrees = int(sd)
        
    if trace:
        print(f'apply interval {scale_degrees} TO  {anote.nameWithOctave} {anote.duration}')
        
    musicScale = state['musicScale']
    note_dur = anote.duration.quarterLength
    scale_note = musicScale.get_note(scale_degrees, anote)
    next_note = copy.deepcopy(scale_note)
    next_note.duration = duration.Duration(note_dur)
    state['previous_note'] = anote
    state['note'] = next_note
    #
    return next_note

def duration_rule(d:str, state):
    '''Apply the duration command d to Note anote.
        Arguments:
            d - a float to multiply the Notes duration.quarterLength by
            anote - a Note
        Note if the resulting product is >4, it is reduced by half.
        If < 0.125, it is multiplied by 2
    '''
    anote = state['note']
    if d=='duration':
        multiplier = 1.0
    else:
        multiplier = float(d)
        
    if trace:
        print(f'apply duration {multiplier} TO {anote.nameWithOctave} {anote.duration}')
        
    old_dur = anote.duration.quarterLength
    new_dur = round(old_dur * multiplier, 3)
    if new_dur > 4.0:      # the maximum duration is a whole note
        new_dur = 2.0    
    elif new_dur < 0.125:  # the minimum duration is 32nd note
        new_dur = 0.25
        
    new_note = note.Note(anote.nameWithOctave, quarterLength=new_dur)
    if trace:
        print(f"\told_dur: {old_dur}  new_dur: {new_dur}\n")
    state['note'] = new_note
    return new_note


In [16]:
import re

substitution_rules = {\
        r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+2/0.5', '-3/1.0', '+1/2.0'], \
        r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['2/duration', '-1/2.0'] #, \
        #r'(?P<interval>[+-]?2)/(?P<duration>\d+\.\d+)' : ['+2/1.0', '+2/2.0, +1/0.5']
}
command_rules = {'interval':interval_rule, 'duration':duration_rule}
rules = {'commands':command_rules}
splitter = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
rule_set = RuleSet(substitution_rules, rules=rules, splitter=splitter)
start = ['0/1.0', '+2/0.5', '-3/0.5', '1/2.0']

ss = MusicSubstitutionSystem(rule_set, verbose=0)
commands = ss.apply(start,1)
print(f'{len(commands)} commands:\n{commands} ')

12 commands:
['0/0.5', '+2/0.5', '-3/1.0', '+1/2.0', '+2/0.5', '-3/0.5', '1/2.0', '0/1.0', '+2/0.5', '-3/0.5', '2/duration', '-1/2.0'] 


In [17]:
score_gen = ScoreGen(rule_set, scale_name='Harmonic Minor', instrument_name='Soprano', key=key.Key('c'), verbose=0)
start_note=note.Note("C5", quarterLength=2)
ascore = score_gen.run(commands, start_note=start_note)


In [18]:
ascore.show('musicxml')

## Test code

In [29]:
# splitter match string
import re

rule_re_str = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
rule_re = re.compile(rule_re_str)
m = rule_re.match('+1/1.0')
gd = m.groupdict()
print(gd)
print(int(gd['interval']))
print(float(gd['duration']))

{'interval': '+1', 'duration': '1.0'}
1
1.0


In [18]:
# match string for specific rule
import re

rule1_re_str = r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)'
rule1_re = re.compile(rule1_re_str)   # re.Pattern
rule2_re_str = r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)'
rule2_re = re.compile(rule2_re_str)
m1 = rule1_re.match('0/0.5')
print(m1.groupdict())

m2 = rule2_re.match('+1/1.0')
if m2 is not None:
    print(m2.groupdict())

{'interval': '0', 'duration': '0.5'}
{'interval': '+1', 'duration': '1.0'}


## Pitch/Duration-based rule set

Rules use pitches (name + octave) and quarterLength durations. Examples:<br>
**'C5/2.0' -->  'E5/1.0', 'B4/0.5', 'D5/0.5', 'C5/1.0'**

Regular expressions allows for matching the pitch only (i.e. no octave).<br>
Grouping Tags are \<pitch\>,  '\<octave\>' and '\<quarterLength\>'
    

In [20]:
import re
#
# Rules may have specific pitches with or without an octave.
#
ms = r'(?P<pitch>[A-Ga-g])(?P<octave>\d?)/(?P<quarterLength>\d+\.\d+)'  # the RuleSet splitter
ms_re = re.compile(ms1)
ms_match = ms_re.match('C4/0.5')
print(ms_match.groupdict())

ms_match = ms_re.match('C/1.0')
print(ms_match.groupdict())  # match any C pitch

{'pitch': 'C', 'octave': '4', 'quarterLength': '0.5'}
{'pitch': 'C', 'octave': '', 'quarterLength': '1.0'}


In [21]:
#
# substitution rules can use discrete pitches or reference group tags
# The ScoreGen rules substitute tags for real values based on the input
#
substitution_rules = {\
    r'(?P<pitch>[A-Ga-g])(?P<octave>\d?)/(?P<quarterLength>\d+\.\d+)' : \
        [ 'pitch+3/quarterLength*0.5', 'pitch-1/quarterLength*0.25', 'pitch+1/quarterLength*0.25', 'pitch/quarterLength*0.5' ],
    r'(?P<pitch>C)(?P<octave>5)/(?P<quarterLength>\d+\.\d+)' : \
        [ 'E5/quarterLength*0.5', 'B4/quarterLength*0.25', 'D5/quarterLength*0.25', 'C5/quarterLength*0.5' ]
}

### Rules Code

In [ ]:
def pitch_rule(sd:str, state):
    anote = state['note']
    

def quarterLength_rule(ql:str, state):
    pass



In [2]:
substitution_rules = {\
    r'(?P<interval>[+-]?[01])/(?P<duration>\d+\.\d+)':['0/0.5', '+3/1.0', '-2/2.0', '-1/2.0']  , \
    r'(?P<interval>[+-]?[23])/(?P<duration>\d+\.\d+)' : ['-1/1.0', '-2/2.0', '+2/0.5', '0/1.0']
}

In [3]:
substitution_rules

{'(?P<interval>[+-]?[01])/(?P<duration>\\d+\\.\\d+)': ['0/0.5',
  '+3/1.0',
  '-2/2.0',
  '-1/2.0'],
 '(?P<interval>[+-]?[23])/(?P<duration>\\d+\\.\\d+)': ['-1/1.0',
  '-2/2.0',
  '+2/0.5',
  '0/1.0']}

In [13]:
import json, math
filename = "resources/music/substitution_rules.json"
fp = open(filename, "r")
jtxt = fp.read()
jdoc = json.loads(jtxt)

In [5]:
jdoc['rule1']

{'(?P<interval>[+-]?[01])/(?P<duration>\\d+\\.\\d+)': ['0/0.5',
  '+3/1.0',
  '-2/2.0',
  '-1/2.0'],
 '(?P<interval>[+-]?[23])/(?P<duration>\\d+\\.\\d+)': ['-1/1.0',
  '-2/2.0',
  '+2/0.5',
  '0/1.0']}

In [18]:
maxlen=26.0
plen = 18.5
math.ceil(maxlen/4) * 4 - plen

9.5

In [23]:
ts = TimeSignature('4/4')

In [24]:
ts.numerator * ts.beatDuration.quarterLength

4.0